In [32]:
import numpy as np
import pandas as pd
import os, itertools, csv
from bs4 import BeautifulSoup
import re
from datetime import *
import time

# load the dataset
dataset = pd.read_csv('./data/test.csv')
dataset.head()

,Id,Page content
0,27643,"<html><head><div class=""article-info""><span cl..."
1,27644,"<html><head><div class=""article-info""><span cl..."
2,27645,"<html><head><div class=""article-info""><span cl..."
3,27646,"<html><head><div class=""article-info""><span cl..."
4,27647,"<html><head><div class=""article-info""><span cl..."


In [2]:
def get_article(text: str) -> str:    
    soup = BeautifulSoup(text, "html.parser")
    article = soup.find('section',{'class':'article-content'})
    return article.get_text()

res = []

for i in range(dataset.shape[0]):
    res.append(get_article(dataset['Page content'][i]))
dataset['article'] = res

In [3]:
dataset.head()

,Id,Page content,article
0,27643,"<html><head><div class=""article-info""><span cl...",Note to humanity: One Direction fandom ai...
1,27644,"<html><head><div class=""article-info""><span cl...",Shortly after announcing a hardware upgrade f...
2,27645,"<html><head><div class=""article-info""><span cl...","Well, that was quick. Just hours after going ..."
3,27646,"<html><head><div class=""article-info""><span cl...",Between Two Ferns: Oscar Buzz Edition Part 1...
4,27647,"<html><head><div class=""article-info""><span cl...",Ever since The Hurt Locker it seems like ...


In [4]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import string
import nltk
from nltk.stem.porter import PorterStemmer

nltk.download('wordnet')
stop_words = set(stopwords.words("english"))

def get_category(text: str) -> list:
    res = list()
    soup = BeautifulSoup(text, "html.parser")
    soup = soup.find('footer',{'class':'article-topics'})
    cats = soup.find_all('a', {'href': re.compile('/category/*')})
    for cat in cats:
        res.append(cat.get_text().lower())
#     porter = PorterStemmer()
#     res = [porter.stem(word) for word in res if not word in  
#         stop_words]
    return list(res)

topics = []
for i in range(0, dataset.shape[0]):
    topics.append(get_category(dataset['Page content'][i]))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yuwei\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
topics[0]

['entertainment', 'music', 'one direction', 'soccer', 'sports']

In [6]:
def get_title(text: str) -> str:    
    soup = BeautifulSoup(text, "html.parser")
    title = soup.find('h1',{'class':'title'})
    return title.get_text()

titles = []
for i in range(0, dataset.shape[0]):
    titles.append(get_title(dataset['Page content'][i]))
# test
# print(get_title(dataset['Page content'][0]))
titles[0]

'Soccer Star Gets Twitter Death Threats After Tackling One Direction Member'

In [7]:
from textblob import TextBlob

TextBlob('Member').intensity

AttributeError: 'TextBlob' object has no attribute 'intensity'

In [3]:
print(text.intensity)

AttributeError: 'TextBlob' object has no attribute 'intensity'

In [7]:
from textblob import TextBlob

n_title_token = []
for i in range(0, dataset.shape[0]):
    n_title_token.append(len(TextBlob(titles[i]).words))

In [8]:
dataset['n_title_token'] = n_title_token

dataset.head()

,Id,Page content,article,n_title_token
0,27643,"<html><head><div class=""article-info""><span cl...",Note to humanity: One Direction fandom ai...,11
1,27644,"<html><head><div class=""article-info""><span cl...",Shortly after announcing a hardware upgrade f...,6
2,27645,"<html><head><div class=""article-info""><span cl...","Well, that was quick. Just hours after going ...",8
3,27646,"<html><head><div class=""article-info""><span cl...",Between Two Ferns: Oscar Buzz Edition Part 1...,6
4,27647,"<html><head><div class=""article-info""><span cl...",Ever since The Hurt Locker it seems like ...,12


In [9]:
# n_article_token = []
# for i in range(0, dataset.shape[0]):
#     n_article_token.append(len(TextBlob(dataset['article'][i]).words))

In [10]:
# dataset['n_article_token'] = n_article_token

# dataset.head()

In [11]:
title_polarity = []
title_subjectivity = []

for i in range(0, dataset.shape[0]):
    testimonial = TextBlob(titles[i])
    title_polarity.append(testimonial.sentiment.polarity)
    title_subjectivity.append(testimonial.sentiment.subjectivity)

In [12]:
dataset['title_polarity'] = title_polarity
dataset['title_subjectivity'] = title_subjectivity

dataset.head()

,Id,Page content,article,n_title_token,title_polarity,title_subjectivity
0,27643,"<html><head><div class=""article-info""><span cl...",Note to humanity: One Direction fandom ai...,11,0.0,0.0
1,27644,"<html><head><div class=""article-info""><span cl...",Shortly after announcing a hardware upgrade f...,6,0.0,0.0
2,27645,"<html><head><div class=""article-info""><span cl...","Well, that was quick. Just hours after going ...",8,0.0,0.0
3,27646,"<html><head><div class=""article-info""><span cl...",Between Two Ferns: Oscar Buzz Edition Part 1...,6,0.0,0.0
4,27647,"<html><head><div class=""article-info""><span cl...",Ever since The Hurt Locker it seems like ...,12,0.0,0.0


In [13]:
article_polarity = []
article_subjectivity = []

for i in range(0, dataset.shape[0]):
    testimonial = TextBlob(dataset["article"][i])
    article_polarity.append(testimonial.sentiment.polarity)
    article_subjectivity.append(testimonial.sentiment.subjectivity)

In [14]:
dataset['article_polarity'] = article_polarity
dataset['article_subjectivity'] = article_subjectivity

dataset.head()

,Id,Page content,article,n_title_token,title_polarity,title_subjectivity,article_polarity,article_subjectivity
0,27643,"<html><head><div class=""article-info""><span cl...",Note to humanity: One Direction fandom ai...,11,0.0,0.0,-0.020425,0.441748
1,27644,"<html><head><div class=""article-info""><span cl...",Shortly after announcing a hardware upgrade f...,6,0.0,0.0,0.094010,0.408221
2,27645,"<html><head><div class=""article-info""><span cl...","Well, that was quick. Just hours after going ...",8,0.0,0.0,0.200833,0.512500
3,27646,"<html><head><div class=""article-info""><span cl...",Between Two Ferns: Oscar Buzz Edition Part 1...,6,0.0,0.0,0.265590,0.462642
4,27647,"<html><head><div class=""article-info""><span cl...",Ever since The Hurt Locker it seems like ...,12,0.0,0.0,0.013742,0.334834


In [15]:
n_topic = []

for i in range(0, dataset.shape[0]):
    n_topic.append(len(topics[i]))

In [16]:
dataset['n_topic'] = n_topic

dataset.head()

,Id,Page content,article,n_title_token,title_polarity,title_subjectivity,article_polarity,article_subjectivity,n_topic
0,27643,"<html><head><div class=""article-info""><span cl...",Note to humanity: One Direction fandom ai...,11,0.0,0.0,-0.020425,0.441748,5
1,27644,"<html><head><div class=""article-info""><span cl...",Shortly after announcing a hardware upgrade f...,6,0.0,0.0,0.094010,0.408221,6
2,27645,"<html><head><div class=""article-info""><span cl...","Well, that was quick. Just hours after going ...",8,0.0,0.0,0.200833,0.512500,4
3,27646,"<html><head><div class=""article-info""><span cl...",Between Two Ferns: Oscar Buzz Edition Part 1...,6,0.0,0.0,0.265590,0.462642,7
4,27647,"<html><head><div class=""article-info""><span cl...",Ever since The Hurt Locker it seems like ...,12,0.0,0.0,0.013742,0.334834,7


In [17]:
channel_list = []

for i in range(0, dataset.shape[0]):
    soup = BeautifulSoup(dataset["Page content"][i], "html.parser")
    article = soup.find('article')
    channel = article.get("data-channel")
    channel_list.append(channel)

In [18]:
dataset['channel'] = channel_list

dataset.head()

,Id,Page content,article,n_title_token,title_polarity,title_subjectivity,article_polarity,article_subjectivity,n_topic,channel
0,27643,"<html><head><div class=""article-info""><span cl...",Note to humanity: One Direction fandom ai...,11,0.0,0.0,-0.020425,0.441748,5,entertainment
1,27644,"<html><head><div class=""article-info""><span cl...",Shortly after announcing a hardware upgrade f...,6,0.0,0.0,0.094010,0.408221,6,tech
2,27645,"<html><head><div class=""article-info""><span cl...","Well, that was quick. Just hours after going ...",8,0.0,0.0,0.200833,0.512500,4,business
3,27646,"<html><head><div class=""article-info""><span cl...",Between Two Ferns: Oscar Buzz Edition Part 1...,6,0.0,0.0,0.265590,0.462642,7,film
4,27647,"<html><head><div class=""article-info""><span cl...",Ever since The Hurt Locker it seems like ...,12,0.0,0.0,0.013742,0.334834,7,entertainment


In [19]:
author = []

for i in range(0, dataset.shape[0]):
    try:
        soup = BeautifulSoup(dataset["Page content"][i], "html.parser")
        soup = soup.find('span', {'class': "author_name"})
        soup = soup.find('a', {'href': re.compile('/author/*')})
        author.append(soup.text)
    except:
        author.append(None)

In [20]:
dataset['author'] = author

dataset.head()

,Id,Page content,article,n_title_token,title_polarity,title_subjectivity,article_polarity,article_subjectivity,n_topic,channel,author
0,27643,"<html><head><div class=""article-info""><span cl...",Note to humanity: One Direction fandom ai...,11,0.0,0.0,-0.020425,0.441748,5,entertainment,Sam Laird
1,27644,"<html><head><div class=""article-info""><span cl...",Shortly after announcing a hardware upgrade f...,6,0.0,0.0,0.094010,0.408221,6,tech,Stan Schroeder
2,27645,"<html><head><div class=""article-info""><span cl...","Well, that was quick. Just hours after going ...",8,0.0,0.0,0.200833,0.512500,4,business,Todd Wasserman
3,27646,"<html><head><div class=""article-info""><span cl...",Between Two Ferns: Oscar Buzz Edition Part 1...,6,0.0,0.0,0.265590,0.462642,7,film,Neha Prakash
4,27647,"<html><head><div class=""article-info""><span cl...",Ever since The Hurt Locker it seems like ...,12,0.0,0.0,0.013742,0.334834,7,entertainment,Josh Dickey


In [21]:
n_image = []

for i in range(0, dataset.shape[0]):
    soup = BeautifulSoup(dataset["Page content"][i], "html.parser")
    article = soup.find('section',{'class':'article-content'})
    imgs = article.find_all('img')
    n_image.append(len(imgs))

In [22]:
dataset['n_image'] = n_image

dataset.head()

,Id,Page content,article,n_title_token,title_polarity,title_subjectivity,article_polarity,article_subjectivity,n_topic,channel,author,n_image
0,27643,"<html><head><div class=""article-info""><span cl...",Note to humanity: One Direction fandom ai...,11,0.0,0.0,-0.020425,0.441748,5,entertainment,Sam Laird,0
1,27644,"<html><head><div class=""article-info""><span cl...",Shortly after announcing a hardware upgrade f...,6,0.0,0.0,0.094010,0.408221,6,tech,Stan Schroeder,1
2,27645,"<html><head><div class=""article-info""><span cl...","Well, that was quick. Just hours after going ...",8,0.0,0.0,0.200833,0.512500,4,business,Todd Wasserman,0
3,27646,"<html><head><div class=""article-info""><span cl...",Between Two Ferns: Oscar Buzz Edition Part 1...,6,0.0,0.0,0.265590,0.462642,7,film,Neha Prakash,0
4,27647,"<html><head><div class=""article-info""><span cl...",Ever since The Hurt Locker it seems like ...,12,0.0,0.0,0.013742,0.334834,7,entertainment,Josh Dickey,0


In [23]:
n_herf = []
n_self_herf = []

for i in range(0, dataset.shape[0]):
    soup = BeautifulSoup(dataset["Page content"][i], "html.parser")
    article = soup.find('section',{'class':'article-content'})
    herfs = article.find_all('a')
    out_herfs = article.find_all('a', {'href': re.compile('^http')})
    n_herf.append(len(herfs))
    n_self_herf.append(len(herfs) - len(out_herfs))

In [24]:
dataset['n_herf'] = n_herf
dataset['n_self_herf'] = n_self_herf

dataset.head()

,Id,Page content,article,n_title_token,title_polarity,title_subjectivity,article_polarity,article_subjectivity,n_topic,channel,author,n_image,n_herf,n_self_herf
0,27643,"<html><head><div class=""article-info""><span cl...",Note to humanity: One Direction fandom ai...,11,0.0,0.0,-0.020425,0.441748,5,entertainment,Sam Laird,0,23,0
1,27644,"<html><head><div class=""article-info""><span cl...",Shortly after announcing a hardware upgrade f...,6,0.0,0.0,0.094010,0.408221,6,tech,Stan Schroeder,1,5,0
2,27645,"<html><head><div class=""article-info""><span cl...","Well, that was quick. Just hours after going ...",8,0.0,0.0,0.200833,0.512500,4,business,Todd Wasserman,0,7,0
3,27646,"<html><head><div class=""article-info""><span cl...",Between Two Ferns: Oscar Buzz Edition Part 1...,6,0.0,0.0,0.265590,0.462642,7,film,Neha Prakash,0,6,0
4,27647,"<html><head><div class=""article-info""><span cl...",Ever since The Hurt Locker it seems like ...,12,0.0,0.0,0.013742,0.334834,7,entertainment,Josh Dickey,0,1,0


In [25]:
def get_weekday(text: str) -> int: #input html format
    soup = BeautifulSoup(text)
    selector = "time"
    date = [i.text for i in soup.select(selector)][0]
    #print(soup.select(selector))
    #print(date)

    try:
        date = date.split()
        date = str(date[0] + ' ' + date[1])
    #print(date)

        d = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
        d = d.weekday()+1
    #print(d)
    except:
        d = 1
        
    return d

weekday = []
for i in range(dataset.shape[0]):
    weekday.append(get_weekday(dataset['Page content'][i]))
    
dataset['weekday'] = weekday

In [26]:
dataset.head()

,Id,Page content,article,n_title_token,title_polarity,title_subjectivity,article_polarity,article_subjectivity,n_topic,channel,author,n_image,n_herf,n_self_herf,weekday
0,27643,"<html><head><div class=""article-info""><span cl...",Note to humanity: One Direction fandom ai...,11,0.0,0.0,-0.020425,0.441748,5,entertainment,Sam Laird,0,23,0,1
1,27644,"<html><head><div class=""article-info""><span cl...",Shortly after announcing a hardware upgrade f...,6,0.0,0.0,0.094010,0.408221,6,tech,Stan Schroeder,1,5,0,4
2,27645,"<html><head><div class=""article-info""><span cl...","Well, that was quick. Just hours after going ...",8,0.0,0.0,0.200833,0.512500,4,business,Todd Wasserman,0,7,0,2
3,27646,"<html><head><div class=""article-info""><span cl...",Between Two Ferns: Oscar Buzz Edition Part 1...,6,0.0,0.0,0.265590,0.462642,7,film,Neha Prakash,0,6,0,3
4,27647,"<html><head><div class=""article-info""><span cl...",Ever since The Hurt Locker it seems like ...,12,0.0,0.0,0.013742,0.334834,7,entertainment,Josh Dickey,0,1,0,5


In [27]:
soup = BeautifulSoup(dataset['Page content'][0])
selector = "time"
date = [i.text for i in soup.select(selector)][0]
date = date.split()
date = str(date[0] + ' ' + date[1])
#print(date)

d = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
print(d)
print(datetime.now())
print(str(datetime.now() - d).split(' ')[0])

2013-09-09 19:47:02
2019-10-29 23:32:05.105643
2241


In [28]:
def get_timedelta(text: str) -> int: #input html format
    soup = BeautifulSoup(text)
    selector = "time"
    date = [i.text for i in soup.select(selector)][0]
    try:
        date = date.split()
        date = str(date[0] + ' ' + date[1])

        d = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
        return int(str(datetime.now() - d).split(' ')[0])
    except:
        return 0

timedelta = []
for i in range(dataset.shape[0]):
    timedelta.append(get_timedelta(dataset['Page content'][i]))
    

In [29]:
dataset['timedelta'] = timedelta
dataset.head()

,Id,Page content,article,n_title_token,title_polarity,title_subjectivity,article_polarity,article_subjectivity,n_topic,channel,author,n_image,n_herf,n_self_herf,weekday,timedelta
0,27643,"<html><head><div class=""article-info""><span cl...",Note to humanity: One Direction fandom ai...,11,0.0,0.0,-0.020425,0.441748,5,entertainment,Sam Laird,0,23,0,1,2241
1,27644,"<html><head><div class=""article-info""><span cl...",Shortly after announcing a hardware upgrade f...,6,0.0,0.0,0.094010,0.408221,6,tech,Stan Schroeder,1,5,0,4,2189
2,27645,"<html><head><div class=""article-info""><span cl...","Well, that was quick. Just hours after going ...",8,0.0,0.0,0.200833,0.512500,4,business,Todd Wasserman,0,7,0,2,2317
3,27646,"<html><head><div class=""article-info""><span cl...",Between Two Ferns: Oscar Buzz Edition Part 1...,6,0.0,0.0,0.265590,0.462642,7,film,Neha Prakash,0,6,0,3,2449
4,27647,"<html><head><div class=""article-info""><span cl...",Ever since The Hurt Locker it seems like ...,12,0.0,0.0,0.013742,0.334834,7,entertainment,Josh Dickey,0,1,0,5,1852


In [30]:
dataset = dataset.drop(columns=["Page content", "article"])
dataset.head()

,Id,n_title_token,title_polarity,title_subjectivity,article_polarity,article_subjectivity,n_topic,channel,author,n_image,n_herf,n_self_herf,weekday,timedelta
0,27643,11,0.0,0.0,-0.020425,0.441748,5,entertainment,Sam Laird,0,23,0,1,2241
1,27644,6,0.0,0.0,0.094010,0.408221,6,tech,Stan Schroeder,1,5,0,4,2189
2,27645,8,0.0,0.0,0.200833,0.512500,4,business,Todd Wasserman,0,7,0,2,2317
3,27646,6,0.0,0.0,0.265590,0.462642,7,film,Neha Prakash,0,6,0,3,2449
4,27647,12,0.0,0.0,0.013742,0.334834,7,entertainment,Josh Dickey,0,1,0,5,1852


In [31]:
dataset.to_csv('./data/new_test.csv')